In [1]:
# dependencies

import cv2
import numpy as np
import pandas as pd
import pickle
import os
import glob
import matplotlib.pyplot as plt

In [2]:
# current directory
os.getcwd()

'/home/nikhil/Desktop/sober_drunk_classification'

In [3]:
# reading images

data_dir = 'mod_database/'

data_cls = ['sober', 'drunk']


# creating complete name for different category of images
def name_gen(name, cat):
    
    name_split = name.split('/')
    
    # modify category
    name_split[2] = cat
    
    # modify in image name
    img_name = name_split[3].split('_')
    img_name[3] = cat[0]
    
    #final rectification
    name_split[3] = '_'.join(img_name)
    
    return '/'.join(name_split)

In [5]:
def get_hog_des() :
    # method to create HOG feature descriptor
    
    winSize = (160, 128) # one descriptor for whole image
    blockSize = (32,32) # 2xcellSize
    blockStride = (16,16) # 50% of blockSize
    cellSize = (16,16)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradient = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradient)

    return hog

In [10]:
hog = get_hog_des()

def get_feature(img_name):
    ''' Perform neccesary tranform and extract features'''
    img = cv2.imread(img_name, cv2.IMREAD_UNCHANGED) # read
    img = cv2.convertScaleAbs(img) # to unint8 from uint16
    ftr = hog.compute(img)
    return ftr
    

In [43]:
feature_set = []

for i, cls in enumerate(data_cls): # label: 0 -> sober, 1 -> drunk
    
    # reading all 4 different images together
    for name_hand in glob.glob(data_dir+cls+'/hand/*'):
        
        try:
            print(name_hand)
            hand_ftr = get_feature(name_hand)

            name_face = name_gen(name_hand, 'face')
            face_ftr = get_feature(name_face)
    #         print(name_face)

            name_ear = name_gen(name_hand, 'ear')
            ear_ftr = get_feature(name_ear)
    #         print(name_ear)

            name_right = name_gen(name_hand, 'right')
            right_ftr = get_feature(name_right)

            # collecting all
            res = np.hstack((face_ftr.T, right_ftr.T, hand_ftr.T, ear_ftr.T))
            comp = np.append(res, i)
            feature_set.append(comp)
            print(res.shape, comp.shape)
        
        except Exception as e:
            print(e)
#         break
    else:
        print(len(feature_set))
#     break

mod_database/sober/hand/06_tris_1_h_F_24_51.tif
(1, 9072) (9073,)
mod_database/sober/hand/31_antonis_1_h_M_45_76.tif
(1, 9072) (9073,)
mod_database/sober/hand/11_takis_1_h_M_20_79.tif
(1, 9072) (9073,)
mod_database/sober/hand/02_giorgos_1_h_M_30_83.tif
(1, 9072) (9073,)
mod_database/sober/hand/17_kwstas_1_h_M_18_63.tif
(1, 9072) (9073,)
mod_database/sober/hand/21_ilias_1_h_M_27_110.tif
(1, 9072) (9073,)
mod_database/sober/hand/37_vassilisA_1_h_M_54_90.tif
(1, 9072) (9073,)
mod_database/sober/hand/28_frag_1_h_M_40_83.tif
(1, 9072) (9073,)
mod_database/sober/hand/01_petros_1_h_M_20_71.tif
(1, 9072) (9073,)
mod_database/sober/hand/19_pasxalis_1_h_M_39_83.tif
(1, 9072) (9073,)
mod_database/sober/hand/16_nikos_1_h_M_18_74.tif
(1, 9072) (9073,)
mod_database/sober/hand/40_spiliop_1_h_M_43_74.tif
(1, 9072) (9073,)
mod_database/sober/hand/15_konstantinos_1_h_M_18_70.tif
(1, 9072) (9073,)
mod_database/sober/hand/08_tzon_1_h_M_32_80.tif
(1, 9072) (9073,)
mod_database/sober/hand/35_thanasis_1_h_M_

In [44]:
# saving the feature data into a csv file 
# total  9072 feature param and 1 label -> 9073 attributes

df  = pd.DataFrame(feature_set)
df.to_csv('ftr.csv', index=False)